In [1]:
import os
import glob
import wave
import IPython.display as ipd
import random
import librosa as lb
import pickle
import pandas as pd
import numpy as np
import multiprocessing
from tqdm import tqdm
from bisect import bisect
from pathlib import Path
from pydub import AudioSegment

/home/ijain/ttmp/anaconda3/envs/mir/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
train_test_folders = {'train': ['Chopin_Op017No4', 'Chopin_Op063No3'],
                      'test': ['Chopin_Op024No2', 'Chopin_Op030No2', 'Chopin_Op068No3']}
EXCLUDE = ['Chopin_Op017No4_Ginzburg-1957_pid9156-10', 'Chopin_Op068No3_Koczalski-1948_pid9140-05']

In [3]:
DATASET = 'test'  # switch to' test' to generate the test data
FOLDERS = train_test_folders[DATASET]

In [4]:
MAIN_DIR = Path('/home/ijain/ttmp/')
BENCHMARKS_DIR = MAIN_DIR/'Chopin_Mazurkas_Benchmarks'
FEATURES_DIR = MAIN_DIR/'Chopin_Mazurkas_features'
AUDIO_TEMP = MAIN_DIR/'Chopin_Mazurkas_audios'

BENCHMARKS_DIR.mkdir(exist_ok=True)
FEATURES_DIR.mkdir(exist_ok=True)
AUDIO_TEMP.mkdir(exist_ok=True)

DATA_DIR = MAIN_DIR/'Chopin_Mazurkas_Modified'
AUDIO_DIR = DATA_DIR/'wav_22050_mono/'
ANNOT_DIR = DATA_DIR/'annotations_beat'

In [5]:
def construct_subseq(x):
    
    with open(BENCHMARKS_DIR/f'{DATASET}_subseq_{x}.log', 'w') as log_file:
        log_file.write('folder, file, start_sec\n')     
        
        for folder in AUDIO_DIR.glob('Chopin*'):
            folder_name = os.path.abspath(folder).split('/')[-1]
            
            # only generate given dataset
            if folder_name not in FOLDERS:
                continue
            
            # create feature and annot directories for folder
            # and audio temp (temp)
            (BENCHMARKS_DIR/f'subseq_{x}'/'annotations_beat'/folder_name).mkdir(parents=True, exist_ok=True)
            (FEATURES_DIR/f'subseq_{x}'/folder_name).mkdir(parents=True, exist_ok=True)
            (AUDIO_TEMP/f'subseq_{x}'/folder_name).mkdir(parents=True, exist_ok=True)
            
            for file in tqdm(folder.glob('*')):
                file_name = os.path.abspath(file).split('/')[-1].split('.')[0]
                
                if file_name in EXCLUDE:
                    continue
                    
                # sample audio and log start sec of sample    
                sample_start, sampled_frames, params = sample_audio(file, x)
                log_file.write(f'{folder_name}, {file_name}, {sample_start}\n')
                
                # compute and save chroma features
                compute_and_save_chroma(sampled_frames, params, Path(f'subseq_{x}')/folder_name/file_name)
            
                create_annot((ANNOT_DIR/folder_name/file_name).with_suffix('.beat'), 
                             (BENCHMARKS_DIR/f'subseq_{x}'/'annotations_beat'/folder_name/file_name).with_suffix('.beat'),
                             sample_start, sample_start+x)
            
    

In [6]:
def construct_partial(partial_type):
    """
        data_path: (Path) Chopin Mazurkas dataset
        partial_type
    """
    
    with open(BENCHMARKS_DIR/f'{DATASET}_partial{partial_type}.log', 'w') as log_file:
        log_file.write('folder, file, percentage\n')
    
        for folder in AUDIO_DIR.glob('Chopin*'):
            folder_name = os.path.abspath(folder).split('/')[-1]
            
            # only generate given dataset
            if folder_name not in FOLDERS:
                continue
            
            # create feature and annot directories for folder
            # and audio temp (temp)
            (BENCHMARKS_DIR/f'partial{partial_type}'/'annotations_beat'/folder_name).mkdir(parents=True, exist_ok=True)
            (FEATURES_DIR/f'partial{partial_type}'/folder_name).mkdir(parents=True, exist_ok=True)
            (AUDIO_TEMP/f'partial{partial_type}'/folder_name).mkdir(parents=True, exist_ok=True)

            for file in tqdm(folder.glob('*')):
                file_name = os.path.abspath(file).split('/')[-1].split('.')[0]
                
                if file_name in EXCLUDE:
                    continue
                
                percentage = random.randint(55, 75)/100
                log_file.write(f'{folder_name}, {file_name}, {percentage}\n')
    
                seconds, sampled_frames, params = sample_percentage(file, percentage, partial_type)
                compute_and_save_chroma(sampled_frames, params, Path(f'partial{partial_type}')/folder_name/file_name)
            
                start_end_dict = {'Start': [0, seconds], 'End': [seconds, None]}
                    
                create_annot((ANNOT_DIR/folder_name/file_name).with_suffix('.beat'),
                             (BENCHMARKS_DIR/f'partial{partial_type}'/'annotations_beat'/folder_name/file_name).with_suffix('.beat'),
                             start_end_dict[partial_type][0], start_end_dict[partial_type][1])
                
                
                

In [7]:
def construct_partial_start():
    construct_partial('Start')
def construct_partial_end():
    construct_partial('End')

In [8]:
def sample_audio(audio_path, x):
    """
        audio_path: (Path) path to audio
        x: (int) length of audio to be samples in seconds
    """
    f = wave.open(os.path.abspath(audio_path), 'rb')
    frames = f.readframes(f.getnframes())
    sample_rate = f.getframerate()
    sample_width = f.getsampwidth()
    params = f.getparams()
    L =  x * sample_rate * sample_width
    start = random.randint(0, (len(frames)-L)//sample_width) * sample_width
    sampled_frames = frames[start:start + L]
    return start/sample_rate/sample_width, sampled_frames, params

In [9]:
def sample_percentage(audio_path, percentage, partial_type):
    f = wave.open(os.path.abspath(audio_path.with_suffix('.wav')), 'rb')
    frames = f.readframes(f.getnframes())
    sample_rate = f.getframerate()
    sample_width = f.getsampwidth()
    params = f.getparams()
    
    if partial_type == 'Start':
        L = int(len(frames) * percentage)
        if L % 2 != 0:
            L += 1
        sampled_frames = frames[:L]
    elif partial_type == 'End':
        L = int(len(frames) * (1 - percentage))
        if L % 2 != 0:
            L += 1
        sampled_frames = frames[L:]
    return L/sample_rate/sample_width, sampled_frames, params

In [10]:
def compute_and_save_chroma(sampled_frames, audio_params, file_path):
    obj = wave.open(os.path.abspath((AUDIO_TEMP/file_path).with_suffix('.wav')), 'wb')
    obj.setparams(audio_params)
    obj.writeframes(sampled_frames)
    obj.close()
    
    y, sr = lb.core.load((AUDIO_TEMP/file_path).with_suffix('.wav'))
    feats = lb.feature.chroma_cqt(y=y, sr=sr, hop_length=512)
    np.save(FEATURES_DIR/file_path, feats)

In [11]:
def get_preamble(file_path):
    with open(file_path, 'r') as f:
        preamble = [f.readline(), 
                    f.readline(),
                    f.readline()]
    return preamble

In [12]:
def create_annot(orig_annot_file, benchmark_annot_file, start=None, end=None):
    preamble = get_preamble(orig_annot_file)
    
    df = pd.read_csv(orig_annot_file, header=None, sep='\s+', skiprows=3)
    
    if start == 0 or start < df[0][0]:
        # partialStart or start sec < first annot sec
        idx=bisect(df[0], end)
        df.at[idx-1,1] = end
        df[0] -= start
        df[1] -= start
        df = df[:idx]
    elif end is None:
        # partialEnd
        idx = bisect(df[0],start)
        if start in df[0]:
            idx = idx - 1
        df[0] -= start
        df[1] -= start
        df = df[idx:len(df[1])-1]
    else:
        # start sec falls on beat
        idx_start = bisect(df[0], start)
        if start in df[0]:
            idx_start = idx_start - 1
        idx_end = bisect(df[0], end)
        df.at[idx_end-1,1] = end
        df[0] -= start
        df[1] -= start
        df = df[idx_start:idx_end]
    
    annots = df.to_csv(None, header=False, index=False, sep='\t')
    
    with open(benchmark_annot_file, 'w') as f:
        f.writelines(preamble)
        f.writelines(annots)

In [13]:
random.seed(42)

In [14]:
construct_subseq(20)

0it [00:00, ?it/s]

64it [08:38,  8.10s/it]
34it [03:31,  6.23s/it]
51it [02:00,  2.37s/it]


In [15]:
construct_subseq(30)

64it [02:34,  2.42s/it]
34it [02:53,  5.11s/it]
51it [05:10,  6.10s/it]


In [16]:
construct_subseq(40)

64it [07:55,  7.43s/it]
34it [05:18,  9.36s/it]
51it [02:38,  3.10s/it]


In [17]:
construct_partial_start()

64it [12:07, 11.37s/it]
34it [06:22, 11.24s/it]
51it [05:38,  6.63s/it]


In [18]:
construct_partial_end()

0it [00:00, ?it/s]

64it [09:49,  9.22s/it]
34it [02:37,  4.63s/it]
51it [07:19,  8.61s/it]


## Make Pre and Post Benchmarks

In [19]:
def compute_chroma_single(infile, outfile, sr = 22050, hop_length=512):
    y, sr = lb.core.load(infile, sr = sr)
    F = lb.feature.chroma_cqt(y=y, sr=sr, hop_length=hop_length)
    np.save(outfile, F)
    return

In [20]:
def construct_pre_post(seconds):
    for folder in AUDIO_DIR.glob('Chopin*'):
        folder_name = os.path.abspath(folder).split('/')[-1]
        
        # only generate given dataset
        if folder_name not in FOLDERS:
            continue
            
        (BENCHMARKS_DIR/f'pre_{seconds}'/'annotations_beat'/folder_name).mkdir(parents=True, exist_ok=True)
        (BENCHMARKS_DIR/f'post_{seconds}'/'annotations_beat'/folder_name).mkdir(parents=True, exist_ok=True)
        
        
        (AUDIO_TEMP/f'pre_{seconds}'/folder_name).mkdir(parents=True, exist_ok=True)
        (AUDIO_TEMP/f'post_{seconds}'/folder_name).mkdir(parents=True, exist_ok=True)
        
        (FEATURES_DIR/f'pre_{seconds}'/folder_name).mkdir(parents=True, exist_ok=True)
        (FEATURES_DIR/f'post_{seconds}'/folder_name).mkdir(parents=True, exist_ok=True)
        
        for file in tqdm(folder.glob('*')):
            file_name = os.path.abspath(file).split('/')[-1].split('.')[0]
            
            if file_name in EXCLUDE:
                continue
            
            final_pre = AudioSegment.silent(duration=seconds*1000) + AudioSegment.from_wav(file)
            final_post = AudioSegment.from_wav(file) + AudioSegment.silent(duration=seconds*1000)
            
            pre_file = os.path.abspath((AUDIO_TEMP/f'pre_{seconds}'/folder_name/file_name).with_suffix('.wav'))
            post_file = os.path.abspath((AUDIO_TEMP/f'post_{seconds}'/folder_name/file_name).with_suffix('.wav'))
            
            final_pre.export(pre_file, format="wav")
            final_post.export(post_file, format="wav")
        
            # compute chromas
            compute_chroma_single(pre_file, FEATURES_DIR/f'pre_{seconds}'/folder_name/file_name)
            compute_chroma_single(post_file, FEATURES_DIR/f'post_{seconds}'/folder_name/file_name)
            
            # annots
            create_annot((ANNOT_DIR/folder_name/file_name).with_suffix('.beat'),
                         (BENCHMARKS_DIR/f'pre_{seconds}'/'annotations_beat'/folder_name/file_name).with_suffix('.beat'),
                        seconds)
            create_annot((ANNOT_DIR/folder_name/file_name).with_suffix('.beat'),
                         (BENCHMARKS_DIR/f'post_{seconds}'/'annotations_beat'/folder_name/file_name).with_suffix('.beat'))
            

In [21]:
def create_annot(orig_annot_file, benchmark_annot_file, pre_seconds=None):
    
    preamble = get_preamble(orig_annot_file)
    df = pd.read_csv(orig_annot_file, header=None, sep='\s+', skiprows=3)
    
    if pre_seconds is None:
        annots = df.to_csv(None, header=False, index=False, sep='\t')
    else:
        df[0] += pre_seconds
        df[1] += pre_seconds

        annots = df.to_csv(None, header=False, index=False, sep='\t')
    
    with open(benchmark_annot_file, 'w') as f:
        f.writelines(preamble)
        f.writelines(annots)
    

In [22]:
construct_pre_post(5)

0it [00:00, ?it/s]

64it [31:34, 29.60s/it]
34it [06:41, 11.82s/it]
51it [12:03, 14.19s/it]


In [23]:
construct_pre_post(10)

64it [30:21, 28.47s/it]
34it [05:18,  9.36s/it]
51it [10:16, 12.08s/it]


In [24]:
construct_pre_post(20)

64it [28:56, 27.13s/it]
34it [11:04, 19.53s/it]
51it [17:09, 20.18s/it]


## Compute and Save Chroma Features for Matching Benchmark

In [25]:
def compute_chroma_batch(filelist, outdir, n_cores):
    
    # prep inputs for parallelization
    inputs = []
    with open(filelist, 'r') as f:
        for line in f:
            relpath = line.strip()
            reldir, fileid = os.path.split(relpath)
            featdir = outdir / reldir
            featdir.mkdir(parents=True, exist_ok=True)
            featfile = (featdir / fileid).with_suffix('.npy')
            audiofile = (AUDIO_DIR / relpath).with_suffix('.wav')
            if os.path.exists(featfile):
                print(f"Skipping {featfile}")
            else:
                inputs.append((audiofile, featfile))

    # process files in parallel
    pool = multiprocessing.Pool(processes = n_cores)
    pool.starmap(compute_chroma_single, inputs)
    
    return

In [26]:
compute_chroma_batch(f'cfg_files/{DATASET}.files.list', FEATURES_DIR / 'matching', 24)

Skipping /home/ijain/ttmp/Chopin_Mazurkas_features/matching/Chopin_Op024No2/Chopin_Op024No2_Hatto-1993_pid610003-17.npy
Skipping /home/ijain/ttmp/Chopin_Mazurkas_features/matching/Chopin_Op024No2/Chopin_Op024No2_Rubinstein-1939_pid9049-15.npy
Skipping /home/ijain/ttmp/Chopin_Mazurkas_features/matching/Chopin_Op024No2/Chopin_Op024No2_Bacha-1998_pid9166e-08.npy
Skipping /home/ijain/ttmp/Chopin_Mazurkas_features/matching/Chopin_Op024No2/Chopin_Op024No2_Nezu-2005_pid9081-03.npy
Skipping /home/ijain/ttmp/Chopin_Mazurkas_features/matching/Chopin_Op024No2/Chopin_Op024No2_Hatto-2006_pid9073-17.npy
Skipping /home/ijain/ttmp/Chopin_Mazurkas_features/matching/Chopin_Op024No2/Chopin_Op024No2_Biret-1990_pid9062-15.npy
Skipping /home/ijain/ttmp/Chopin_Mazurkas_features/matching/Chopin_Op024No2/Chopin_Op024No2_Mohovich-1999_pid9104-15.npy
Skipping /home/ijain/ttmp/Chopin_Mazurkas_features/matching/Chopin_Op024No2/Chopin_Op024No2_Fiorentino-1961_pid9065-10.npy
Skipping /home/ijain/ttmp/Chopin_Mazurkas

In [27]:
! mkdir -p Chopin_Mazurkas_features/original

In [28]:
! cp -r Chopin_Mazurkas_features/matching/* Chopin_Mazurkas_features/original

cp: cannot stat 'Chopin_Mazurkas_features/matching/*': No such file or directory
